<a href="https://colab.research.google.com/github/endnef2/big_leader/blob/main/Task_Documentation/NLP_%EA%B3%BC%EC%A0%9C_D4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 전처리(불용어)

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN
from tensorflow.keras.utils import to_categorical
import numpy as np
from string import punctuation
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

##1. 데이터 가져오기

In [62]:
news = fetch_20newsgroups(subset='train')

In [63]:
df = pd.DataFrame(news.data, columns=['articles'])

In [64]:
test = fetch_20newsgroups(subset='test', shuffle=True) # shuffle : 데이터 섞기
test

len(test) # 5
len(test.data) # 7532

7532

In [65]:
df['target'] = news.target

In [66]:
df

,articles,target
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14
...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,13
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,4
11311,From: westes@netcom.com (Will Estes)\nSubject:...,3
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,1


In [67]:
test = fetch_20newsgroups(subset='test', shuffle=True)

In [68]:
train_article = df['articles']
train_target = df['target']

In [69]:
test_article = test.data
test_target = test.target

## 2. 데이터 전처리

### 2-1. 불용어 제거
- 길이 2 이하, 소문자로 변환, 불용어사전에 있는 단어 제외

In [70]:
def remove_stopwords(text_list):
    stop_words = set(stopwords.words('english'))
    filtered_texts = []
    for text in text_list:
        words = word_tokenize(text)  # 텍스트를 단어 단위로 토큰화
        filtered_text = [word for word in words if len(word) > 2 and word.lower() not in stop_words and not any(c in word for c in "'s" + punctuation)]
        filtered_texts.append(' '.join(filtered_text))
    return filtered_texts

### 2-2. 데이터 전처리

In [71]:
def pre_processing_data(trainData, testData, ty): # (트레인, 테스트, 유형분류)
    # 데이터 전처리
    trainData = remove_stopwords(trainData)
    testData = remove_stopwords(testData)
    tok = Tokenizer(num_words=10000) # 10000 차원 벡터 공간에 데이터를 임베딩하겠다.
    tok.fit_on_texts(trainData) # trainData에 번호 부여
    xtrain = tok.texts_to_matrix(trainData, mode=ty) # 행렬로 변환해라
    xtest = tok.texts_to_matrix(testData, mode=ty) # 행렬로 변환해라
    return xtrain, xtest, tok.index_word

In [72]:
# 함수 적용
xtrain, xtest, indexWord = pre_processing_data(train_article, test_article, "binary")

In [73]:
xtrain.shape

(11314, 10000)

In [74]:
xtest.shape

(7532, 10000)

In [75]:
len(indexWord)

69738

In [76]:
indexWord

{1: 'subject',
 2: 'organization',
 3: 'would',
 4: 'one',
 5: 'article',
 6: 'people',
 7: 'like',
 8: 'know',
 9: 'get',
 10: 'max',
 11: 'think',
 12: 'new',
 13: 'time',
 14: 'could',
 15: 'good',
 16: 'may',
 17: 'even',
 18: 'well',
 19: 'way',
 20: 'many',
 21: 'god',
 22: 'two',
 23: 'make',
 24: 'much',
 25: 'right',
 26: 'want',
 27: 'world',
 28: 'anyone',
 29: 'need',
 30: 'work',
 31: 'problem',
 32: 'really',
 33: 'computer',
 34: 'believe',
 35: 'going',
 36: 'back',
 37: 'file',
 38: 'information',
 39: 'help',
 40: 'find',
 41: 'take',
 42: 'year',
 43: 'point',
 44: 'better',
 45: 'never',
 46: 'government',
 47: 'might',
 48: 'another',
 49: 'number',
 50: 'got',
 51: 'without',
 52: 'program',
 53: 'read',
 54: 'david',
 55: 'drive',
 56: 'made',
 57: 'data',
 58: 'available',
 59: 'let',
 60: 'thing',
 61: 'look',
 62: 'part',
 63: 'little',
 64: 'etc',
 65: 'come',
 66: 'power',
 67: 'law',
 68: 'however',
 69: 'long',
 70: 'anything',
 71: 'around',
 72: 'key',
 

In [77]:
train_target

0         7
1         4
2         4
3         1
4        14
         ..
11309    13
11310     4
11311     3
11312     1
11313     8
Name: target, Length: 11314, dtype: int64

In [78]:
# one-hot-encoding
ytrain = to_categorical(train_target, 20)
ytest = to_categorical(test_target, 20)

In [79]:
ytrain.shape

(11314, 20)

In [80]:
xtrain.shape

(11314, 10000)

## 신경망 모델

In [81]:
#딥러닝 모델 설계
model=Sequential()
model.add(Dense(256, input_shape=(10000,), activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(20, activation='softmax'))
#모델링 환경 변수 설정
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])
#모델링(트레이닝)
model.fit(xtrain, ytrain, batch_size=128, epochs=5, verbose=1,
          validation_split=0.1)

Epoch 1/5
80/80 [==============================] - 2s 10ms/step - loss: 1.5271 - accuracy: 0.6781 - val_loss: 0.6063 - val_accuracy: 0.8481
Epoch 2/5
80/80 [==============================] - 1s 7ms/step - loss: 0.2669 - accuracy: 0.9462 - val_loss: 0.4566 - val_accuracy: 0.8763
Epoch 3/5
80/80 [==============================] - 1s 7ms/step - loss: 0.0894 - accuracy: 0.9872 - val_loss: 0.4596 - val_accuracy: 0.8710
Epoch 4/5
80/80 [==============================] - 1s 7ms/step - loss: 0.0404 - accuracy: 0.9958 - val_loss: 0.4593 - val_accuracy: 0.8710
Epoch 5/5
80/80 [==============================] - 1s 8ms/step - loss: 0.0226 - accuracy: 0.9977 - val_loss: 0.4699 - val_accuracy: 0.8657


In [82]:
score=model.evaluate(xtest,ytest,batch_size=128,verbose=0)
print(score)

[0.907025933265686, 0.7719065546989441]


# 2. 모델 구조 변경 및 성능 개선

In [83]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
from string import punctuation

In [84]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/NYT_2018.csv')

In [85]:
headline=[]
headline.extend(list(df.headline.values))

### Unknown 처리하기


In [86]:
# Unknown 처리하기
len(headline)

1324

In [87]:
headline

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?',
 'Commuter Reprogramming',
 'Unknown',
 'Unknown',
 'Ford Changed Leaders, Looking for a Lift. It’s Still Looking.',
 'Romney Failed to Win at Utah Convention, But Few Believe He’s Doomed',
 'Chain Reaction',
 'He Forced the Vatican to Investigate Sex Abuse. Now He’s Meeting With Pope Francis.',
 'In Berlin, artists find a home',
 'Unknown',
 'The Right Stuff',
 'Jimmy Carter Knows What North Korea Wants',
 'The Truth Is Out There',
 'New Jersey Ruling Could Reignite Battle Over Church-State Separation',
 'Procrastinating',
 'Word + Quiz: dilatory',
 'My Life-Threatening Bout With E. Coli Food Poisoning',
 'Choosing Brexit, a Town Yearned for Its Seafa

In [88]:
len(df[df['headline']=='Unknown'])

110

In [89]:
sum(df.headline=='Unknown')

110

In [90]:
len([w for w in headline if w=='Unknown'])

110

In [91]:
headline = [w for w in headline if w!='Unknown']

headline에 저장된 뉴스 기사 제목으로 다음 단어를 생성하는 LSTM 기반 모델 설계

동작방식 예) 입력: I, 생성하고자 하는 단어의 개수 -> 출력 : I was ... 완성

In [92]:
# 전처리
headline[10]

'In Berlin, artists find a home'

In [93]:
def pre_func(title):
   # 소문자 변환
   res = ''.join(w.lower() for w in title if w not in punctuation) # 문장 하나의 단어들을 소문자 변환 후 join으로 합쳐서 문장으로 저장
   return res

In [94]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [95]:
pre_headline=[pre_func(x) for x in headline] # 리스트의 요소로 pre_func(x) 함수의 return 값이 들어감

In [96]:
pre_headline[10]

'in berlin artists find a home'

In [97]:
len(pre_headline)

1214

In [98]:
pre_headline

['former nfl cheerleaders’ settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression',
 'commuter reprogramming',
 'ford changed leaders looking for a lift it’s still looking',
 'romney failed to win at utah convention but few believe he’s doomed',
 'chain reaction',
 'he forced the vatican to investigate sex abuse now he’s meeting with pope francis',
 'in berlin artists find a home',
 'the right stuff',
 'jimmy carter knows what north korea wants',
 'the truth is out there',
 'new jersey ruling could reignite battle over churchstate separation',
 'procrastinating',
 'word  quiz dilatory',
 'my lifethreatening bout with e coli food poisoning',
 'choosing brexit a town yearned for its seafaring past and muddied its future',
 'a quote disproved',
 'hot stuff turns cold',
 'at the top the pay gap may be gone',
 '

In [99]:
tok=Tokenizer()

In [100]:
tok.fit_on_texts(pre_headline)

In [101]:
len(tok.index_word) #단어 개수

3619

In [102]:
tok.index_word

{1: 'the',
 2: 'a',
 3: 'to',
 4: 'of',
 5: 'in',
 6: 'for',
 7: 'and',
 8: 'on',
 9: 'is',
 10: 'with',
 11: 'trump',
 12: 'as',
 13: 'at',
 14: 'new',
 15: 'from',
 16: 'how',
 17: 'it',
 18: 'an',
 19: 'be',
 20: 'that',
 21: 'season',
 22: 'us',
 23: 'you',
 24: 'what',
 25: 'episode',
 26: 'can',
 27: 'your',
 28: 'he',
 29: 'not',
 30: 'his',
 31: 'are',
 32: 'teaching',
 33: 'out',
 34: 'was',
 35: 'no',
 36: 'by',
 37: 'trump’s',
 38: 'has',
 39: 'war',
 40: 'over',
 41: 'may',
 42: 'into',
 43: 'more',
 44: 'now',
 45: 'why',
 46: 'who',
 47: 'about',
 48: 'recap',
 49: 'activities',
 50: '1',
 51: 'we',
 52: 'do',
 53: 'women',
 54: 'when',
 55: 'syria',
 56: 'trade',
 57: 'it’s',
 58: '2',
 59: 'or',
 60: 'will',
 61: 'i',
 62: 'have',
 63: 'just',
 64: 'but',
 65: 'home',
 66: 'up',
 67: 'this',
 68: 'one',
 69: 'off',
 70: 'house',
 71: 'president',
 72: 'gop',
 73: 'our',
 74: 'long',
 75: 'facebook',
 76: 'case',
 77: 'life',
 78: 'end',
 79: 'some',
 80: 'power',
 81: '

### 인코딩

In [103]:
# 인코딩, 정수형태 변환
sequences=[]
for s in pre_headline:
    # print(tok.texts_to_sequences([s])[0])
    enc = tok.texts_to_sequences([s])[0] # 각 문장별 인코딩
    for i in range(1, len(enc)):
        seq = enc[:i+1]
        sequences.append(seq)

In [104]:
print(max(len(i) for i in sequences))

24


In [105]:
vocab_size = len(tok.index_word)+1 # 3619개

In [106]:
len(tok.word_index)

3619

In [107]:
print(max(len(i) for i in sequences))
ml = max(len(i) for i in sequences)

24


### padding

In [108]:
sequences=pad_sequences(sequences, maxlen=ml, padding='pre') # 단어의 길이를 지정한 길이로 설정

In [109]:
# sequences의 전체 문장들에 대해 맨 마지막의 숫자 하나만 y에 넣어주는 코드 작성
type(sequences)

numpy.ndarray

In [110]:
y = sequences[:,-1]
x = sequences[:,:-1]

### one-hot encoding

In [111]:
y = to_categorical(y,vocab_size)

In [112]:
# one hot encoding 해주기! => cost 함수가 계산해서 확률로 나오기 위해 이 작업 필요
vocab_size

3620

In [113]:
model = Sequential()
model.add(Embedding(vocab_size, 10))  # 3620 차원 -> 10 차원
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))

In [114]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 10)          36200     
                                                                 
 lstm_3 (LSTM)               (None, 128)               71168     
                                                                 
 dense_7 (Dense)             (None, 3620)              466980    
                                                                 
Total params: 574,348
Trainable params: 574,348
Non-trainable params: 0
_________________________________________________________________


### 환경설정

In [115]:
# 환경설정
model.compile(loss="categorical_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x,y,epochs=200, verbose=2)

Epoch 1/200
245/245 - 12s - loss: 7.6939 - accuracy: 0.0242 - 12s/epoch - 49ms/step
Epoch 2/200
245/245 - 2s - loss: 7.1558 - accuracy: 0.0291 - 2s/epoch - 9ms/step
Epoch 3/200
245/245 - 2s - loss: 7.0140 - accuracy: 0.0353 - 2s/epoch - 8ms/step
Epoch 4/200
245/245 - 2s - loss: 6.8872 - accuracy: 0.0428 - 2s/epoch - 7ms/step
Epoch 5/200
245/245 - 2s - loss: 6.7360 - accuracy: 0.0449 - 2s/epoch - 7ms/step
Epoch 6/200
245/245 - 1s - loss: 6.5666 - accuracy: 0.0483 - 1s/epoch - 6ms/step
Epoch 7/200
245/245 - 1s - loss: 6.3804 - accuracy: 0.0544 - 1s/epoch - 6ms/step
Epoch 8/200
245/245 - 2s - loss: 6.1880 - accuracy: 0.0578 - 2s/epoch - 8ms/step
Epoch 9/200
245/245 - 2s - loss: 5.9865 - accuracy: 0.0611 - 2s/epoch - 9ms/step
Epoch 10/200
245/245 - 1s - loss: 5.7899 - accuracy: 0.0693 - 1s/epoch - 5ms/step
Epoch 11/200
245/245 - 1s - loss: 5.6060 - accuracy: 0.0709 - 1s/epoch - 6ms/step
Epoch 12/200
245/245 - 1s - loss: 5.4293 - accuracy: 0.0789 - 1s/epoch - 5ms/step
Epoch 13/200
245/245 -

In [116]:
def gen_sent(model, tok, c_word, n):
    pred_sent=''
    for _ in range(n):
        enc= tok.texts_to_sequences([c_word])[0]
        # print(enc)
        enc = pad_sequences([enc], maxlen=ml, padding='pre')
        # print(enc)
        res=model.predict(enc)
        res=np.argmax(res)

        for w, i in tok.word_index.items():
            if i==res:
                break
        print("예측단어:", w) # The new
        c_word = c_word+ " "+ w # The => The new
    print(c_word)
    # pred_sent=
    # return pred_sent

In [117]:
gen_sent(model, tok, 'the', 10)

1/1 [==============================] - 0s 333ms/step
예측단어: new
1/1 [==============================] - 0s 19ms/step
예측단어: shingles
1/1 [==============================] - 0s 17ms/step
예측단어: vaccine
1/1 [==============================] - 0s 17ms/step
예측단어: reasons
1/1 [==============================] - 0s 21ms/step
예측단어: to
1/1 [==============================] - 0s 21ms/step
예측단어: get
1/1 [==============================] - 0s 20ms/step
예측단어: it
1/1 [==============================] - 0s 19ms/step
예측단어: gop
1/1 [==============================] - 0s 19ms/step
예측단어: from
1/1 [==============================] - 0s 20ms/step
예측단어: it
the new shingles vaccine reasons to get it gop from it


In [119]:
# model = Sequential()
# model.add(Embedding(vocab_size, 20))  # 3620 차원 -> 10 차원
# model.add(LSTM(128))
# model.add(LSTM(64))
# model.add(Dense(vocab_size, activation='softmax'))

In [120]:
# # 환경설정
# model.compile(loss="categorical_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])
# model.fit(x,y,epochs=200, verbose=2)